In [ ]:
! pip install pyngrok
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.4 MB/s eta 0:00:00


In [34]:
#!pip install streamlit pyngrok tensorflow==2.12.0 joblib plotly
#!pip install yfinance --upgrade
#!pip install protobuf==3.20.3 tensorflow==2.12.0 yfinance==0.2.28 --force-reinstall

from google.colab import files
import os

# Nettoyage initial
!rm -f crypto_* app.py

# Upload des fichiers
uploaded = files.upload()

# Renommage automatique des fichiers
for filename in list(uploaded.keys()):
    if '(' in filename:
        new_name = filename.split('(')[0].strip() + filename.split(')')[-1]
        os.rename(filename, new_name)
        print(f"Fichier renommé : {filename} → {new_name}")

# Vérification des fichiers requis
REQUIRED_FILES = ['crypto_predictor.keras', 'crypto_processor.joblib']
missing = [f for f in REQUIRED_FILES if not os.path.exists(f)]
if missing:
    raise FileNotFoundError(f"Fichiers manquants : {', '.join(missing)}")

# Création du fichier app.py
APP_CODE = """
import streamlit as st
import re
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime
import sqlite3
import requests
from tensorflow.keras.models import load_model
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

# Configuration
MODEL_PATH = 'crypto_predictor.keras'
PROCESSOR_PATH = 'crypto_processor.joblib'
SEQ_LENGTH = 60

# Initialisation BDD
conn = sqlite3.connect('users.db', check_same_thread=False)
c = conn.cursor()

# Création des tables
c.execute('''CREATE TABLE IF NOT EXISTS users
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              username TEXT UNIQUE,
              password TEXT)''')

c.execute('''CREATE TABLE IF NOT EXISTS comments
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              username TEXT,
              comment TEXT,
              timestamp DATETIME)''')

c.execute('''CREATE TABLE IF NOT EXISTS alerts
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              user_id INTEGER,
              condition TEXT,
              value REAL,
              created_at DATETIME)''')
conn.commit()

# Fonctions d'authentification
def create_user(username, password):
    try:
        c.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False

def verify_user(username, password):
    c.execute('SELECT * FROM users WHERE username=? AND password=?', (username, password))
    return c.fetchone() is not None

# Récupération des données
def fetch_real_data():
    # Données Bitcoin
    btc = yf.Ticker("BTC-USD")
    hist = btc.history(period="2y").tz_localize(None)

    # Fear & Greed Index
    response = requests.get("https://api.alternative.me/fng/?limit=730")
    fng_data = pd.DataFrame(response.json()['data'])

    fng_data['timestamp'] = pd.to_datetime(fng_data['timestamp'], unit='s', utc=True)
    fng_data.set_index('timestamp', inplace=True)
    fng_data.index = fng_data.index.tz_localize(None)

    full_data = hist.join(fng_data[['value']].rename(columns={'value': 'fear_greed'}), how='left')
    return full_data.ffill().rename(columns={'Fear_Greed': 'fear_greed'})

# Calcul du RSI
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50)  # Remplissage des NaN avec 50 (valeur neutre)

# Traitement des caractéristiques
def process_features(df):
    df = df.copy()
    df['Volatility'] = (df['High'] - df['Low']) / df['Low'].rolling(24).mean()
    df['Momentum'] = df['Close'].pct_change(periods=12)
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    return df[['Close', 'High', 'Low', 'Volume', 'Volatility', 'Momentum', 'EMA_20']].dropna()

# Visualisation
def plot_close_with_details(data):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Close'],
        name='Close',
        line=dict(color='#d62728'),
        hovertemplate=None
    ))

    features = data.columns.tolist()
    fig.data[0].customdata = data.values

    hovertemplate = "<b>%{x|%d %B %Y %H:%M}</b><br>"
    for i, col in enumerate(features):
        if col == 'Close':
            hovertemplate += f"{col}: %{{y:.2f}}<br>"
        else:
            if np.issubdtype(data[col].dtype, np.integer):
                hovertemplate += f"{col}: %{{customdata[{i}]:.0f}}<br>"
            else:
                hovertemplate += f"{col}: %{{customdata[{i}]:.2f}}<br>"

    hovertemplate += "<extra></extra>"
    fig.update_traces(hovertemplate=hovertemplate)

    fig.update_layout(
        xaxis=dict(rangeslider=dict(visible=True)),
        yaxis_title='Prix (USD)',
        hovermode="x unified",
        height=500
    )
    return fig

class CryptoDataProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, feature_cols=['High', 'Low', 'Volume', 'Volatility', 'Momentum', 'EMA_20'],
                 target_col='Close'):
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.scaler = RobustScaler() # avant StandardScaler()...................
        self.target_idx = 6

    def fit(self, X, y=None):
        self.scaler.fit(X[self.feature_cols])
        return self

    def transform(self, X):
        return self.scaler.transform(X[self.feature_cols])

    def inverse_transform(self, X):
        X = np.array(X)
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)

        full_matrix = np.zeros((X.shape[0], 7))
        full_matrix[:, self.target_idx] = X.flatten()

        return self.scaler.inverse_transform(full_matrix)[:, self.target_idx]

class BitcoinChatbot:
    def __init__(self, model, processor, seq_length):
        self.model = model
        self.processor = processor
        self.seq_length = seq_length
        self.thresholds = {
            "rsi_buy": 30,
            "rsi_sell": 70,
            "fear_greed_buy": 25,
            "fear_greed_sell": 75,
            "volatility_high": 0.05
        }
        self.mae = self.calculate_recent_mae(window=30) or 1552

    def ensure_features(self, df):
        required_features = ['Close', 'High', 'Low', 'Volume', 'Volatility', 'Momentum', 'EMA_20']

        if 'Volatility' not in df.columns:
            df['Volatility'] = (df['High'] - df['Low']) / df['Low'].rolling(24).mean()
        if 'Momentum' not in df.columns:
            df['Momentum'] = df['Close'].pct_change(periods=12)
        if 'EMA_20' not in df.columns:
            df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()

        return df[required_features]

    def process_features(self, df):
        df = df.copy()
        df['Volatility'] = (df['High'] - df['Low']) / df['Low'].rolling(24).mean()
        df['Momentum'] = df['Close'].pct_change(periods=12)
        df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
        return df[['Close', 'High', 'Low', 'Volume', 'Volatility', 'Momentum', 'EMA_20']].dropna()

    def calculate_recent_mae(self, window=30):
        try:
            btc = yf.Ticker("BTC-USD")
            hist = btc.history(period=f"{self.seq_length + window + 1}d")

            processed_data = self.process_features(hist)
            scaled_data = self.processor.transform(processed_data)

            sequences = np.array([scaled_data[i:i+self.seq_length]
                               for i in range(len(scaled_data)-self.seq_length-window)])

            preds = self.model.predict(sequences[-window:])
            actuals = scaled_data[self.seq_length:-window, self.processor.target_idx]

            pred_prices = self.processor.inverse_transform(preds.flatten())
            actual_prices = self.processor.inverse_transform(actuals)
            return mean_absolute_error(actual_prices, pred_prices)
        except Exception as e:
            st.error(f"Erreur calcul MAE: {str(e)}")
            return None

    def get_market_data(self):
        data = {}
        btc = yf.Ticker("BTC-USD")
        hist = btc.history(period="7d", interval="1h")

        # Données de base
        data["price"] = hist["Close"].iloc[-1]
        data["change_24h"] = hist["Close"].pct_change(24).iloc[-1] * 100

        # Calcul du RSI
        delta = hist["Close"].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(14).mean().iloc[-1]
        avg_loss = loss.rolling(14).mean().iloc[-1]
        data["rsi"] = 100 - (100 / (1 + (avg_gain / avg_loss))) if avg_loss != 0 else 50

        # Fear & Greed Index
        try:
            response = requests.get("https://api.alternative.me/fng/?limit=1")
            data["fear_greed"] = int(response.json()["data"][0]["value"])
        except:
            data["fear_greed"] = 50

        # Volatilité
        data["volatility"] = (hist["High"] - hist["Low"]).mean() / hist["Low"].mean()

        return data

    def predict_tomorrow_price(self):
        if not self.model:
            return None

        try:
            btc = yf.Ticker("BTC-USD")
            hist = btc.history(period=f"{self.seq_length+1}d")

            processed_data = self.process_features(hist)
            scaled_data = self.processor.transform(processed_data)

            last_sequence = scaled_data[-self.seq_length:]
            prediction = self.model.predict(np.array([last_sequence])).flatten()[0]
            return self.processor.inverse_transform([prediction])[0]
        except Exception as e:
            st.error(f"Erreur prédiction: {str(e)}")
            return None

    def generate_advice(self):
        market_data = self.get_market_data()
        advice = []

        # Analyse Volatilité
        if market_data['volatility'] > self.thresholds['volatility_high']:
            advice.append("⚠️ **Volatilité élevée** - Marché instable, soyez prudent")

        # Analyse RSI
        if market_data['rsi'] < self.thresholds['rsi_buy']:
            advice.append("✅ **RSI bas** - Bonne opportunité d'achat")
        elif market_data['rsi'] > self.thresholds['rsi_sell']:
            advice.append("⚠️ **RSI élevé** - Possible sur-achat")
        else:
            advice.append("➖ **RSI neutre** - Pas de signal fort")

        # Analyse Fear & Greed
        if market_data['fear_greed'] < self.thresholds['fear_greed_buy']:
            advice.append("✅ **Fear & Greed: Peur extrême** - Opportunité d'achat")
        elif market_data['fear_greed'] > self.thresholds['fear_greed_sell']:
            advice.append("⚠️ **Fear & Greed: Avarice extrême** - Possible correction")

        return advice

    def generate_response(self, user_input):
        user_input = user_input.lower()

        if re.search(r"(prix demain|prediction demain|tomorrow)", user_input):
            if not self.model:
                st.error("❌ Le modèle de prédiction n'est pas chargé")
                return

            tomorrow_price = self.predict_tomorrow_price()
            if tomorrow_price:
                current_price = self.get_market_data()["price"]
                change_pct = (tomorrow_price - current_price) / current_price * 100
                arrow = "↑" if change_pct >= 0 else "↓"
                confidence = "élevée" if self.mae < 1000 else "moyenne" if self.mae < 2000 else "faible"

                # Utilisation de containers et colonnes natifs
                with st.container():
                    st.subheader(f"🔮 Prévision Bitcoin (Confiance: {confidence})")

                    col1, col2 = st.columns([1, 3])
                    with col1: st.write("Prix actuel:")
                    with col2: st.write(f"{current_price:.2f} USD")

                    col1, col2 = st.columns([1, 3])
                    with col1: st.write("Prévision demain:")
                    with col2: st.write(f"{tomorrow_price:.2f} USD ({arrow}{abs(change_pct):.2f}%)")

                    col1, col2 = st.columns([1, 3])
                    with col1: st.write("Fourchette:")
                    with col2: st.write(f"{tomorrow_price-self.mae:.2f} à {tomorrow_price+self.mae:.2f} USD")

                    col1, col2 = st.columns([1, 3])
                    with col1: st.write("Précision:")
                    with col2: st.write(f"±{self.mae:.2f} USD (30 jours)")

            else:
                st.error("❌ Impossible de générer une prédiction")

        elif re.search(r"(conseil|acheter|vendre|signal)", user_input):
            advice = self.generate_advice()
            market_data = self.get_market_data()

            with st.container():
                st.subheader("💡 Conseil Trading")

                col1, col2 = st.columns([1, 3])
                with col1: st.write("Prix actuel:")
                with col2: st.write(f"{market_data['price']:.2f} USD")

                col1, col2 = st.columns([1, 3])
                with col1: st.write("Précision:")
                with col2: st.write(f"±{self.mae:.2f} USD")

                st.subheader("📌 Recommandations")
                for item in advice:
                    st.write(f"• {item}")

        elif re.search(r"(indicateurs|metrics|rsi|volatilité)", user_input):
            data = self.get_market_data()

            with st.container():
                st.subheader("📊 Indicateurs Clés")

                col1, col2 = st.columns(2)

                with col1:
                    st.write(f"💰 Prix: {data['price']:.2f} USD")
                    st.write(f"📈 Variation des dernières 24h: {data['change_24h']:.2f}%")
                    st.write(f"📉 RSI: {data['rsi']:.1f}")

                with col2:
                    st.write(f"😨 Fear & Greed: {data['fear_greed']}")
                    st.write(f"⚡ Volatilité: {data['volatility']*100:.1f}%")
                    st.write(f"🎯 MAE modèle: ±{self.mae:.2f} USD")

# Interface utilisateur
def main():
    st.title("📈 Crypto Analytics Pro+")

    # Gestion authentification
    if 'authenticated' not in st.session_state:
        st.session_state.authenticated = False
        st.session_state.username = None
        st.session_state.chatbot = None  # Initialize chatbot in session state

    if not st.session_state.authenticated:
        auth_type = st.radio("Action", ["Connexion", "Création compte"])
        username = st.text_input("Nom d'utilisateur")
        password = st.text_input("Mot de passe", type="password")

        if st.button("Valider"):
            if auth_type == "Connexion":
                if verify_user(username, password):
                    st.session_state.authenticated = True
                    st.session_state.username = username
                    # Initialize chatbot after successful login
                    model = load_model(MODEL_PATH, compile=False)
                    processor = joblib.load(PROCESSOR_PATH)
                    st.session_state.chatbot = BitcoinChatbot(model, processor, SEQ_LENGTH)
                    st.rerun()
                else:
                    st.error("Identifiants incorrects")
            else:
                if create_user(username, password):
                    st.success("Compte créé avec succès!")
                else:
                    st.error("Ce nom d'utilisateur existe déjà")
        return

    # Menu principal
    st.sidebar.header(f"Bienvenue {st.session_state.username}")

    # Section Chatbot
    with st.sidebar.expander("💬 Assistant Bitcoin", expanded=True):
        user_query = st.text_input("Posez une question...", key="chatbot_input")
        if user_query and st.session_state.chatbot:
            st.session_state.chatbot.generate_response(user_query)

    # Section données
    st.header("Analyse des données")
    data_source = st.radio("Source des données", ["Fichier CSV", "Données temps réel"])

    if data_source == "Fichier CSV":
        uploaded_file = st.file_uploader("Choisir un fichier CSV", type="csv")
        data = pd.read_csv(uploaded_file, index_col='Date', parse_dates=True) if uploaded_file else None
    else:
        data = fetch_real_data()

    if data is None:
        return

    # Traitement des données
    processed_data = process_features(data[['Close', 'High', 'Low', 'Volume']])

    # Visualisation
    st.subheader("Analyse technique complète")
    st.plotly_chart(plot_close_with_details(processed_data))

    # Section prédiction
    st.header("Prédiction des prix")
    model = load_model(MODEL_PATH, compile=False)
    processor = joblib.load(PROCESSOR_PATH)

    if st.button("Exécuter la prédiction"):
        scaled_data = processor.scaler.transform(processed_data)

        # Création séquences
        sequences = np.array([scaled_data[i:i+SEQ_LENGTH]
                            for i in range(len(scaled_data)-SEQ_LENGTH)])

        # Prédiction
        pred = model.predict(sequences).flatten()
        pred_prices = processor.inverse_transform(pred)
        actual_prices = processor.inverse_transform(scaled_data[SEQ_LENGTH:, processor.target_idx])

        # Calcul des métriques
        mae = mean_absolute_error(actual_prices, pred_prices)

        # Calcul de la précision directionnelle
        direction_real = np.diff(actual_prices)
        direction_pred = np.diff(pred_prices)
        directional_acc = np.mean((direction_real * direction_pred) > 0) * 100

        # Calcul du MACD
        exp1 = processed_data['Close'].ewm(span=12, adjust=False).mean()
        exp2 = processed_data['Close'].ewm(span=26, adjust=False).mean()
        MACD = exp1 - exp2
        latest_macd = MACD.iloc[-1]

        # Prédiction du lendemain
        last_sequence = scaled_data[-SEQ_LENGTH:]
        next_day_pred = model.predict(np.array([last_sequence])).flatten()[0]
        next_day_price = processor.inverse_transform([next_day_pred])[0]

        # Calcul du RSI (14 périodes)
        rsi = calculate_rsi(pd.Series(actual_prices)).iloc[-1]

        # Dernière valeur de volatilité
        latest_volatility = processed_data['Volatility'].iloc[-1] * 100  # Conversion en pourcentage

        # Affichage des métriques
        col1, col2, col3, col4, col5 = st.columns(5)
        with col1:
            st.metric("MAE", f"{mae:.2f} $", help="Erreur Moyenne Absolue")
        with col2:
            st.metric("MACD", f"{latest_macd:.2f}",
                    help="Force du momentum")
        with col3:
            st.metric("Prévision J+1", f"{next_day_price:.2f} $",
                    help="Prédiction du prix pour le prochain jour")
        with col4:
            st.metric("RSI Actuel", f"{rsi:.1f}",
                    help="Indice de Force Relative (14 périodes)")
        with col5:
            st.metric("Volatilité", f"{latest_volatility:.2f}%",
                    help="Volatilité moyenne sur 24 heures")

        # Visualisation
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=processed_data.index[SEQ_LENGTH:],
            y=actual_prices,
            name='Réel',
            line=dict(color='#1f77b4')
        ))
        fig.add_trace(go.Scatter(
            x=processed_data.index[SEQ_LENGTH:],
            y=pred_prices,
            name='Prédiction',
            line=dict(color='#FF5733', dash='dot')
        ))
        fig.update_layout(
            height=500,
            title="Comparaison des prix",
            hovermode="x unified",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        st.plotly_chart(fig)

    # Section commentaires
    st.header("Espace communautaire")
    new_comment = st.text_area("Nouveau commentaire")
    if st.button("Publier"):
        c.execute('INSERT INTO comments (username, comment, timestamp) VALUES (?, ?, ?)',
                 (st.session_state.username, new_comment, datetime.now()))
        conn.commit()

    # Affichage commentaires
    comments = c.execute('''SELECT username, comment, timestamp
                          FROM comments ORDER BY timestamp DESC LIMIT 20''').fetchall()
    for user, comment, timestamp in comments:
        st.markdown(f"**{user}** (*{datetime.strptime(str(timestamp), '%Y-%m-%d %H:%M:%S.%f').strftime('%d/%m/%Y %H:%M')}*) : {comment}")

    # Gestion alertes
    with st.sidebar.expander("🔔 Gestion des Alertes", expanded=True):
        # Création de nouvelles alertes
        new_condition = st.selectbox("Type d'alerte", [
            "Prix >", "Prix <",
            "RSI > 70", "RSI < 30",
            "Fear & Greed > 75", "Fear & Greed < 25"
        ])
        new_value = st.number_input("Seuil", value=0.0, step=0.1)

        if st.button("➕ Créer une nouvelle alerte"):
            c.execute('SELECT id FROM users WHERE username=?', (st.session_state.username,))
            user_id = c.fetchone()[0]
            c.execute('''INSERT INTO alerts (user_id, condition, value, created_at)
                       VALUES (?, ?, ?, ?)''',
                     (user_id, new_condition, new_value, datetime.now()))
            conn.commit()
            st.success("Alerte créée avec succès!")

        # Suppression des alertes existantes
        st.markdown("---")
        st.subheader("Alertes Actives")
        alerts = c.execute('''SELECT id, condition, value
                            FROM alerts
                            WHERE user_id = (SELECT id FROM users WHERE username=?)''',
                          (st.session_state.username,)).fetchall()

        if not alerts:
            st.write("Aucune alerte active")
        else:
            for alert_id, cond, val in alerts:
                cols = st.columns([4, 1])
                with cols[0]:
                    st.markdown(f"`{cond} {val}`")
                with cols[1]:
                    if st.button("🗑️", key=f"del_{alert_id}"):
                        c.execute('DELETE FROM alerts WHERE id = ?', (alert_id,))
                        conn.commit()
                        st.rerun()

    # Affichage alertes
    st.sidebar.subheader("Alertes actives")
    alerts = c.execute('''SELECT condition, value FROM alerts
               WHERE user_id = (SELECT id FROM users WHERE username=?)''',
             (st.session_state.username,)).fetchall()
    for cond, val in alerts:
        st.sidebar.markdown(f"⚠️ **{cond} {val}**")

if __name__ == "__main__":
    main()
"""

with open('app.py', 'w') as f:
    f.write(APP_CODE)

# Démarrage de l'application
from pyngrok import ngrok
ngrok.set_auth_token("2wKDAWnB6VJX2Rx7tAp9q2U5FvL_qhuyK4HYQy3rdHc5xfF6")

public_url = ngrok.connect(8501).public_url
print(f"\n✅ Application disponible à : {public_url}\n")

!streamlit run --server.port 8501 app.py



Saving crypto_processor.joblib to crypto_processor.joblib
Saving crypto_predictor.keras to crypto_predictor.keras

✅ Application disponible à : https://9806-34-46-3-238.ngrok-free.app




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.46.3.238:8501

2025-05-19 00:31:55.159560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747614715.191870   30698 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747614715.200683   30698 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 00:32:13.629627: E external/local_xla/xla/stream_executor/cuda/cuda